In [1]:
import os
import openai
import time
import boto3, json, os
from botocore.config import Config

os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.environ["OPENAI_API_KEY"]

os.environ['AWS_PROFILE'] = ""
os.environ['AWS_DEFAULT_REGION'] = "us-east-1"

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from llama_index.core import SimpleDirectoryReader, get_response_synthesizer, StorageContext
from llama_index.core import DocumentSummaryIndex
from llama_index.llms.openai import OpenAI
from llama_index.core.node_parser import SentenceSplitter

### Ingesting data

In [ ]:
## Ingesting documents from Local


data_root = "./juergen_data/"

filenames = ["capturing-the-full-value-of-generative-ai-in-banking",
             "Industry4_BigBets_Oct25"]
#            "Industry4_Framework_10_03",
#            "JuergenLindner_Resume_Jan2024",
#            "State_of_Industry_4.0_Research_Overview_2022",
#            "technologys-generational-moment-with-generative-ai-a-cio-and-cto-guide"]

docs = []

for filename in filenames:
    doc = SimpleDirectoryReader(input_files=[f"{data_root}/{filename}.pdf"]).load_data()
    doc[0].doc_id = filename.replace(".pdf","")
    docs.extend(doc)

## Run the Redis-Based Ingestion Pipeline

In [ ]:
from llama_index.storage.kvstore.redis import RedisKVStore as RedisCache
from llama_index.storage.docstore.redis import RedisDocumentStore
from llama_index.core.node_parser import SentenceSplitter
from llama_index.vector_stores.redis import RedisVectorStore
from llama_index.storage.index_store.redis import RedisIndexStore

In [ ]:
llm = OpenAI(model="gpt-3.5-turbo-0125", temperature=0)
text_splitter = SentenceSplitter(chunk_size=3000)
embed_model = OpenAI(model="text-embedding-3-small")

### Setting up vector store

In [ ]:
REDIS_HOST = ''
REDIS_PORT = 6379

In [ ]:
from sqlalchemy import make_url
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.postgres import PGVectorStore

In [ ]:
url = make_url("postgres://postgres.<Username>:<Password>@aws-0-us-west-1.pooler.supabase.com:5432/postgres")
db_name="postgres"
vector_store = PGVectorStore.from_params(
    database=url.database,
    host=url.host,
    password=url.password,
    port=url.port,
    user=url.username,
    table_name="test-llamaindex-aiva",
    embed_dim=1536,  # openai embedding dimension
)

### Create connection to Redis cache

In [ ]:

from llama_index.core import SimpleDirectoryReader

storage_context = StorageContext.from_defaults(
    docstore=RedisDocumentStore.from_host_and_port(
        host=REDIS_HOST, port=REDIS_PORT, namespace="redis-document-store"
    ),
    
    index_store=RedisIndexStore.from_host_and_port(
        host=REDIS_HOST, port=REDIS_PORT, namespace="redis-index-store"
    ),
    vector_store = vector_store
)

response_synthesizer = get_response_synthesizer(
    response_mode = "tree_summarize", use_async=True
)

In [ ]:
doc_summary_index = DocumentSummaryIndex.from_documents(docs,
    llm=llm,
    transformations=[text_splitter],
    response_synthesizer=response_synthesizer,
    show_progress=True,
    storage_context = storage_context)

In [ ]:
from llama_index.core import PromptTemplate
from IPython.display import Markdown, display

def display_prompt_dict(prompts_dict):
    for k, p in prompts_dict.items():
        text_md = f"**Prompt Key**: {k}<br>" f"**Text:** <br>"
        display(Markdown(text_md))
        print(p.get_template())
        display(Markdown("<br><br>"))

new_summary_tmpl_str = (
    "Context information is below:\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge,"
    "answer the query in the style of a McKinsey, Bain or BCG consultant who specializes in digital transformation strategies."
    "Your goal is to help business users succeed in their digital transformation journeys."
    "Do not use any context outside of these documents."
    "If a question is outside of your area of expertise, politely refuse to answer and suggest alternative topics of discussion from the context provided."
    "You should maintain a friendly yet professional tone."
    "Use detailed bullet points whenever relevant.\n"
    "Query: {query_str}\n"
    "Answer: "
)

new_summary_tmpl = PromptTemplate(new_summary_tmpl_str)

In [ ]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.indices.document_summary import (
    DocumentSummaryIndexEmbeddingRetriever,
)

from llama_index.core import load_indices_from_storage

## Uncomment during inference
#doc_summary_index = load_indices_from_storage(
#    storage_context=storage_context
    #index_id=index_id
#)

# Configuring response synthesizer
response_synthesizer = get_response_synthesizer(streaming=True, response_mode="tree_summarize")

## Creating Retreiver object
retriever = DocumentSummaryIndexEmbeddingRetriever(
    doc_summary_index,
    choice_batch_size=10,
    choice_top_k=5
)

# Assembling query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer
)

## Checking default query prompt:
prompts_dict = query_engine.get_prompts()
display_prompt_dict(prompts_dict)

## Modifying query prompt

query_engine.update_prompts(
    {"response_synthesizer:summary_template": new_summary_tmpl}
)

## Checking modified query prompt:
prompts_dict = query_engine.get_prompts()
display_prompt_dict(prompts_dict)

### Embeddings Retirever

In [ ]:
start_time = time.time()
# query
response = query_engine.query("Given me a detailed explanation about the likely impact of Industry 4.0 digital transformation strategies?")
response.print_response_stream()

## Use streaming response in block:
'''for text in streaming_response.response_gen:
    # do something with text as they arrive.
    pass
'''

print("--- %s seconds ---" % (time.time() - start_time))